In [9]:
import configparser
import psycopg2

# Load configuration
config = configparser.ConfigParser()
config.read('dwh.cfg')

# Database connection details
HOST = config.get("CLUSTER", "HOST")
DB_NAME = config.get("CLUSTER", "DB_NAME")
DB_USER = config.get("CLUSTER", "DB_USER")
DB_PASSWORD = config.get("CLUSTER", "DB_PASSWORD")
DB_PORT = config.get("CLUSTER", "DB_PORT")
DB_CONNECTION = f"host={HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} port={DB_PORT}"

# Establish a connection
conn = psycopg2.connect(DB_CONNECTION)
cur = conn.cursor()


In [10]:
# Print the total number of events and songs in staging
cur.execute("SELECT COUNT(*) FROM staging_events;")
results = cur.fetchone()
print(f"Total number of events in staging: {results[0]}")

cur.execute("SELECT COUNT(*) FROM staging_songs;")
results = cur.fetchone()
print(f"Total number of songs in staging: {results[0]}")


Total number of events in staging: 8056
Total number of songs in staging: 385252


In [11]:
# Print the most played song
cur.execute("""
    SELECT a.name, s.title, COUNT(*) as plays
    FROM songplays p
    JOIN artists a ON p.artist_id = a.artist_id
    JOIN songs s ON s.song_id = p.song_id
    GROUP BY a.name, s.title
    ORDER BY plays DESC LIMIT 1;
""")
results = cur.fetchone()
print(f"Most played song with {results[2]} plays is '{results[1]}' by {results[0]}")

Most played song with 110 plays is 'Greece 2000' by Three Drives


In [12]:
# Print the most active location
cur.execute("""
    SELECT location, COUNT(*) as plays
    FROM songplays
    GROUP BY location
    ORDER BY plays DESC LIMIT 1;
""")
results = cur.fetchone()
print(f"Most active location with {results[1]} plays is {results[0]}")

Most active location with 722 plays is San Francisco-Oakland-Hayward, CA


In [13]:
# 20 top listeners and their locations
cur.execute("""
    SELECT user_id, location, COUNT(songplay_id) AS number_of_plays
    FROM songplays
    GROUP BY user_id, location
    ORDER BY number_of_plays DESC, location, user_id
    LIMIT 20;
""")
print("Top 20 listeners and their locations:")
for row in cur.fetchall():
    print(row)

Top 20 listeners and their locations:
(49, 'San Francisco-Oakland-Hayward, CA', 720)
(80, 'Portland-South Portland, ME', 710)
(97, 'Lansing-East Lansing, MI', 581)
(15, 'Chicago-Naperville-Elgin, IL-IN-WI', 508)
(44, 'Waterloo-Cedar Falls, IA', 438)
(29, 'Atlanta-Sandy Springs-Roswell, GA', 368)
(24, 'Lake Havasu City-Kingman, AZ', 339)
(73, 'Tampa-St. Petersburg-Clearwater, FL', 312)
(36, 'Janesville-Beloit, WI', 275)
(88, 'Sacramento--Roseville--Arden-Arcade, CA', 275)
(16, 'Birmingham-Hoover, AL', 248)
(95, 'Winston-Salem, NC', 225)
(85, 'Red Bluff, CA', 191)
(30, 'San Jose-Sunnyvale-Santa Clara, CA', 190)
(25, 'Marinette, WI-MI', 183)
(58, 'Augusta-Richmond County, GA-SC', 146)
(42, 'New York-Newark-Jersey City, NY-NJ-PA', 144)
(26, 'San Jose-Sunnyvale-Santa Clara, CA', 124)
(82, 'Atlanta-Sandy Springs-Roswell, GA', 81)
(72, 'Detroit-Warren-Dearborn, MI', 77)


In [14]:
# 50 top songs
cur.execute("""
    SELECT s.title, a.name AS artist, COUNT(sp.songplay_id) AS number_of_plays
    FROM songplays sp
    JOIN songs s ON sp.song_id = s.song_id
    JOIN artists a ON sp.artist_id = a.artist_id
    GROUP BY s.title, a.name
    ORDER BY number_of_plays DESC, a.name, s.title
    LIMIT 50;
""")
print("Top 50 songs:")
for row in cur.fetchall():
    print(row)

Top 50 songs:
('Greece 2000', 'Three Drives', 110)
('Stronger', 'Kanye West', 84)
('Greece 2000', '3 Drives On A Vinyl', 55)
('This Fire', 'Franz Ferdinand', 54)
('Yellow', 'Coldplay', 48)
('Stronger', "Kanye West / Consequence / Cam'Ron", 42)
('Stronger', 'Kanye West / GLC / Consequence', 42)
('Stronger', 'Kanye West / Jay-Z', 42)
('Stronger', 'Kanye West / Jay-Z / J. Ivy', 42)
('Stronger', 'Kanye West / John Legend / Consequence', 42)
('Stronger', 'Kanye West / Kid Cudi', 42)
('Stronger', 'Kanye West / Lil Wayne', 42)
('Stronger', 'Kanye West / Mos Def', 42)
('Stronger', 'Kanye West / Mos Def / Al Be Back', 42)
('Stronger', 'Kanye West / Mos Def / Freeway / The Boys Choir Of Harlem', 42)
('Stronger', 'Kanye West / Nas / Really Doe', 42)
('Stronger', 'Kanye West / Paul Wall / GLC', 42)
('Stronger', 'Kanye West / Syleena Johnson', 42)
('Stronger', 'Kanye West / T-Pain', 42)
('Stronger', 'Kanye West / Talib Kweli / Q-Tip / Common / Rhymefest', 42)
('Stronger', 'Kanye West / Twista / Key

In [15]:
# Songs per duration
cur.execute("""
    SELECT 
        CASE
            WHEN duration < 60 THEN '0-59 Seconds'
            WHEN duration < 120 THEN '60-119 Seconds'
            WHEN duration < 180 THEN '120-179 Seconds'
            WHEN duration < 240 THEN '180-239 Seconds'
            WHEN duration < 300 THEN '240-299 Seconds'
            ELSE '300+ Seconds'
        END AS duration_range,
        COUNT(*) AS songs
    FROM songs
    GROUP BY 1
    ORDER BY 1;
""")
print("Songs per duration range:")
for row in cur.fetchall():
    print(row)

Songs per duration range:
('0-59 Seconds', 5408)
('120-179 Seconds', 62094)
('180-239 Seconds', 126835)
('240-299 Seconds', 92783)
('300+ Seconds', 80567)
('60-119 Seconds', 17308)


In [16]:
# Close connection
cur.close()
conn.close()